<a href="https://colab.research.google.com/github/IgnasiOliveras/anonimitzar/blob/main/2_arxius_JOIN_def.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd

# Cargar el archivo principal
df_principal = pd.read_excel('/content/30.01.25 Uic.xlsx', sheet_name='Export')

# Filtrar solo ContactId 466 y 300
df_principal_subset = df_principal[df_principal['ContactId'].isin([466, 300])]

# Convertir a datetime
df_principal_subset["fecha"] = pd.to_datetime(df_principal_subset["createdAt"], format="%d/%m/%Y").dt.date

# Calcular rango de 10 segundos
start_time = df_principal_subset["createdAt"].dt.floor('10S')
end_time = start_time + pd.Timedelta(seconds=10)

df_principal_subset["rango"] = (
    start_time.dt.strftime("%H:%M:%S") + "-" + end_time.dt.strftime("%H:%M:%S")
)

# Cargar el archivo de fichas CRM
df_fichas_crm = pd.read_excel('/content/20.02.25FichasCrm (2).xlsx')

# Filtrar solo ContactId 466 y 300
df_fichas_crm_subset = df_fichas_crm[df_fichas_crm['ContactId'].isin([466, 300])]

# Convertir fechas y renombrar columna ContactId
df_fichas_crm_subset["fecha"] = pd.to_datetime(df_fichas_crm_subset["Data/hora inici"], errors="coerce", dayfirst=True)
df_fichas_crm_subset["fecha_str"] = df_fichas_crm_subset["fecha"].dt.strftime("%d/%m/%Y")
df_fichas_crm_subset = df_fichas_crm_subset.rename(columns={'Número de telèfon de l’usuari': 'ContactId2'})

# Calcular rango de 10 segundos
start_time = df_fichas_crm_subset["fecha"].dt.floor('10S')
end_time = start_time + pd.Timedelta(seconds=10)

df_fichas_crm_subset["rango2"] = (
    start_time.dt.strftime("%H:%M:%S") + "-" + end_time.dt.strftime("%H:%M:%S")
)

# Cargar el archivo de tipificaciones
df_tipificaciones = pd.read_excel('/content/20.02.25TipificacionesConversaciones.xlsx')

# Filtrar solo ContactId 466 y 300
df_tipificaciones_subset = df_tipificaciones[df_tipificaciones['ContactId'].isin([466, 300])]

# Renombrar columnas
df_tipificaciones_subset = df_tipificaciones_subset.rename(columns={'ContactId': 'ContactId1'})

# Realizar el primer LEFT JOIN entre df_principal_subset y df_tipificaciones_subset
df_merged = pd.merge(
    df_principal_subset,
    df_tipificaciones_subset,
    how='left',
    left_on=['OpenchannelInteractionId', 'ContactId'],
    right_on=['id', 'ContactId1']
)


df_merged2 = pd.merge(
    df_principal_subset,
    df_fichas_crm_subset,
    how='left',
    left_on=['ContactId'],
    right_on=[ 'ContactId']

)


df_merged.to_excel('df_merged.xlsx', index=False)
df_merged2.to_excel('df_merged2.xlsx', index=False)


import pandas as pd

# Cargar los archivos
df_merged = pd.read_excel("/content/df_merged.xlsx")
df_merged2 = pd.read_excel("/content/df_merged2.xlsx")

# --- Asegurar formatos consistentes ---
# Convertir todas las fechas a datetime
df_merged["fecha"] = pd.to_datetime(df_merged["fecha"])
# Changed line: specify the correct format for 'fecha_str'
df_merged2["fecha_str"] = pd.to_datetime(df_merged2["fecha_str"], format="%d/%m/%Y")

# Normalizar rangos de tiempo (ej: "14:30:20-14:30:30")
df_merged2["rango2"] = df_merged2["rango2"].str.replace(" ", "").str.strip()

# --- Realizar el JOIN con columnas alineadas ---
# Changed lines: Rename 'rango' in df_merged and 'rango2' in df_merged2 to avoid duplicate column names
df_final = pd.merge(
    df_merged.rename(columns={"rango": "rango_merged"}),
    df_merged2.rename(columns={
        "fecha_str": "fecha",  # Renombrar columna para igualar nombres
        "rango2": "rango_merged2"
    }),
    left_on=["fecha", "rango_merged", "ContactId"],
    right_on=["fecha", "rango_merged2", "ContactId"],  # Ahora las columnas tienen los mismos nombres
    how="left"
)

# Guardar en una ruta válida
df_final.to_excel("/content/df_final.xlsx", index=False)

# Mostrar resultado
print("Filas en df_final:", len(df_final))
print(df_final.head())


<ipython-input-18-2ea13fc57934>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_principal_subset["fecha"] = pd.to_datetime(df_principal_subset["createdAt"], format="%d/%m/%Y").dt.date
<ipython-input-18-2ea13fc57934>:13: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  start_time = df_principal_subset["createdAt"].dt.floor('10S')
<ipython-input-18-2ea13fc57934>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_principal_subset["rango"] = (
<ipython-i

Filas en df_final: 7055
    id_x body_x  read_x  secret_x direction_x            readAt_x  \
0  77623   Hola       1         0          in 2023-01-01 00:11:06   
1  77623   Hola       1         0          in 2023-01-01 00:11:06   
2  77623   Hola       1         0          in 2023-01-01 00:11:06   
3  77623   Hola       1         0          in 2023-01-01 00:11:06   
4  77623   Hola       1         0          in 2023-01-01 00:11:06   

   providerName_x  providerResponse_x         createdAt_x         updatedAt_x  \
0             NaN                 NaN 2023-01-01 00:10:19 2023-01-01 00:11:06   
1             NaN                 NaN 2023-01-01 00:10:19 2023-01-01 00:11:06   
2             NaN                 NaN 2023-01-01 00:10:19 2023-01-01 00:11:06   
3             NaN                 NaN 2023-01-01 00:10:19 2023-01-01 00:11:06   
4             NaN                 NaN 2023-01-01 00:10:19 2023-01-01 00:11:06   

   ...  Vinculació: S’ha ofert informació d’algun recurs  \
0  ...        

In [9]:
def procesar_datos(df_principal, df_fichas_crm):
    """
    Realiza el procesamiento principal de los datos

    Args:
        df_principal (DataFrame): DataFrame principal
        df_fichas_crm (DataFrame): DataFrame de fichas CRM

    Returns:
        DataFrame: DataFrame final procesado
    """
    try:
        # Calcular rangos para ambos DataFrames
        df_principal = calcular_rango(df_principal, "createdAt", "rango_principal")
        df_fichas_crm = calcular_rango(df_fichas_crm, "fecha", "rango_fichas")

        # Realizar JOINs
        df_merged = pd.merge(
            df_principal,
            df_tipificaciones,
            how='left',
            left_on=['OpenchannelInteractionId', 'ContactId'],
            right_on=['id', 'ContactId']
        )

        df_merged2 = pd.merge(
            df_principal,
            df_fichas_crm,
            how='left',
            left_on=['ContactId'],
            right_on=['ContactId']
        )

        # Guardar resultados intermedios
        df_merged.to_excel('/content/df_merged.xlsx', index=False)
        df_merged2.to_excel('/content/df_merged2.xlsx', index=False)

        # Cargar archivos intermedios
        df_merged = pd.read_excel("/content/df_merged.xlsx")
        df_merged2 = pd.read_excel("/content/df_merged2.xlsx")

        # Normalizar formatos
        df_merged["fecha"] = pd.to_datetime(df_merged["fecha"])
        df_merged2["fecha_str"] = pd.to_datetime(df_merged2["fecha_str"], format="%d/%m/%Y")
        df_merged2["rango_fichas"] = df_merged2["rango_fichas"].str.replace(" ", "").str.strip()

        # JOIN final con columnas específicas
        df_final = pd.merge(
            df_merged,
            df_merged2,
            left_on=["fecha", "rango_principal", "ContactId"],
            right_on=["fecha", "rango_fichas", "ContactId"],
            how="left"
        )

        # Eliminar columnas duplicadas
        df_final = df_final.loc[:, ~df_final.columns.duplicated()]

        return df_final

    except Exception as e:
        print(f"Error en el procesamiento de datos: {str(e)}")
        raise

In [11]:
def eliminar_columnas_duplicadas(df):
    """
    Elimina las columnas duplicadas de un DataFrame, manteniendo solo la primera ocurrencia

    Args:
        df (DataFrame): DataFrame del cual eliminar las columnas duplicadas

    Returns:
        DataFrame: DataFrame con las columnas duplicadas eliminadas
    """
    return df.loc[:, ~df.columns.duplicated()]


    # Después de crear df_final
df_final = eliminar_columnas_duplicadas(df_final)

print("Columnas después de eliminar duplicados:")
print(df_final.columns.tolist())

Columnas después de eliminar duplicados:
['id_x', 'body_x', 'read_x', 'secret_x', 'direction_x', 'readAt_x', 'providerName_x', 'providerResponse_x', 'createdAt_x', 'updatedAt_x', 'OpenchannelAccountId_x', 'OpenchannelInteractionId_x', 'UserId_x', 'ContactId', 'AttachmentId_x', 'integrationId_x', 'parentIntegrationId_x', 'sentBy_x', 'fecha', 'rango_merged', 'id_y', 'closed', 'closedAt', 'disposition', 'note', 'ContactId1', 'id', 'body_y', 'read_y', 'secret_y', 'direction_y', 'readAt_y', 'providerName_y', 'providerResponse_y', 'createdAt_y', 'updatedAt_y', 'OpenchannelAccountId_y', 'OpenchannelInteractionId_y', 'UserId_y', 'AttachmentId_y', 'integrationId_y', 'parentIntegrationId_y', 'sentBy_y', 'fecha_x', 'rango', 'ID fitxa', 'ID voluntari / Nom voluntari', 'Data/hora inici', 'Data/hora final', 'És un xat en blanc?', 'Unnamed: 6', 'Projecte', 'Edat', 'Gènere', 'País de naixement', 'Barri/Districte', 'Població', 'Província', 'Comunitat Autònoma', 'Trucada de fora d’Espanya', 'Unitat de c

In [20]:
# Renombrar columnas eliminando sufijos _x e _y
#df_final.columns = df_final.columns.str.replace('_x', '').str.replace('_y', '')

# Volver a eliminar duplicados
#df_final = df_final.loc[:, ~df_final.columns.duplicated()]

# Eliminar las columnas usando índices
df_final = df_final.iloc[:, 20:]

# Verificar las columnas restantes
print("Columnas restantes en df_final:")
print(df_final.columns.tolist())
df_final = df_final.drop_duplicates(subset=['body_y'], keep='first')

# Ver el resultado
print("\nColumnas después de eliminar sufijos y duplicados:")
print(df_final.columns.tolist())

print("\nPrimeras filas del DataFrame:")
print(df_final.head())


# Crear el fichero Excel
df_final.to_excel('/content/df_final.xlsx', index=False)

print("Fichero Excel creado correctamente")

Columnas restantes en df_final:
['id_y', 'closed', 'closedAt', 'disposition', 'note', 'ContactId1', 'id', 'body_y', 'read_y', 'secret_y', 'direction_y', 'readAt_y', 'providerName_y', 'providerResponse_y', 'createdAt_y', 'updatedAt_y', 'OpenchannelAccountId_y', 'OpenchannelInteractionId_y', 'UserId_y', 'AttachmentId_y', 'integrationId_y', 'parentIntegrationId_y', 'sentBy_y', 'fecha_x', 'rango', 'ID fitxa', 'ID voluntari / Nom voluntari', 'Data/hora inici', 'Data/hora final', 'És un xat en blanc?', 'Unnamed: 6', 'Projecte', 'Edat', 'Gènere', 'País de naixement', 'Barri/Districte', 'Població', 'Província', 'Comunitat Autònoma', 'Trucada de fora d’Espanya', 'Unitat de convivència', 'Nivell d’estudis', 'Situació laboral', 'Problemàtica', 'Altres', 'Block Ideació suïcida: Carrera suïcida', 'Block Ideació suïcida: Ha pensat que seria millor morir-se o ha desitjat estar mort?', 'Block Ideació suïcida: Ha volgut fer-se mal?', 'Block Ideació suïcida: Ha pensat en el suïcidi?', 'Block Ideació suï